In [1]:
import Pkg
Pkg.add("Pkg")
Pkg.add("Dates")

cd("/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/")
Pkg.activate(".")

using DataFrames, JLD, Distributions, CSV, Random, Plots,JSON
import DataStructures: PriorityQueue, enqueue!, dequeue!

turnaround = Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 40
ncalls = 1000
#using AmbulanceDeployment

pwd()

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%                                      22.9%                                    39.6%
  Resolving package versions...
  Installed FFMPEG ────────── v0.3.0
  Installed PlotThemes ────── v1.0.3
  Installed PlotUtils ─────── v0.6.5
  Installed Plots ─────────── v1.0.14
  Installed FileIO ────────── v1.6.4
  Installed ForwardDiff ───── v0.10.17
  Installed Gtk ───────────── v1.1.7
  Installed RecipesPipeline ─ v0.1.13
  Installed GeometryTypes ─── v0.8.4
  Installed GR ────────────── v0.48.0
Updating `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Project.toml`
  [44cfe95a] + Pkg
Updating `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Manifest.toml`
  [621f4979] ↑ AbstractFFTs v1.0.0 ⇒ v1.0.1
  [79e6a3ab] ↑ Adapt v2.3.0 ⇒ v3.2.0
  [4fba245c] - ArrayInterface v2.14.13
  [336ed68f] ↑ CSV v0.8.2 ⇒ v0.8.4
  [9a962f9c] ↑ DataAPI v1

"/Users/guyfarmer/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"

In [3]:
cd("test/")
pwd()

"/Users/guyfarmer/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test"

In [4]:
#using AmbulanceDeployment
using Dates

In [7]:
include("..//src//model.jl")
include("..//src//dispatch/closestdispatch.jl")
include("..//src//problem.jl")
include("..//src//evaluate.jl")
include("..//src//simulate.jl")

  Resolving package versions...
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`
  Resolving package versions...
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`


LoadError: LoadError: syntax: incomplete: "function" at /Users/guyfarmer/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/src/simulate.jl:50 requires end
in expression starting at /Users/guyfarmer/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/src/simulate.jl:50

In [78]:
hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)

In [79]:
p = DeploymentProblem(
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )
test_calls = CSV.File("../test/austin-data/austin_test_calls.csv")|> DataFrame
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)

Random.seed!(1234)
@time df, guiArray = simulate_events!(problem, dispatch)